# Chapter 18. asyncio를 이용한 동시성

* 동시성은 구조에 관한 것으로서 한 번에 많은 것을 다룬다. 
* 병렬성은 실행에 관한 것으로서 한 번에 많은 것을 한다.

이 장에서는 이벤트 루프에 의해 운용되는 코루틴을 이용해서 동시성을 구현하는 `asyncio` 패키지에 대해 설명한다. 

* 간단한 스레드 프로그램과 그에 준하는 `asyncio` 버전을 비교하면서 스레드와 비동기 작업의 관계를 보여준다.
* `asyncio.Future` 클래스와 `concurrent.futures.Future` 클래스의 차이점을 설명한다.
* 17장에서 구현한 국기 내려받기 예제의 비동기 버전을 구현한다.
* 스레드나 프로세스를 사용하지 않고 비동기 프로그래밍이 네트워크 프로그램에서 높은 동시성을 관리하는 방법을 설명한다.
* 코루틴으로 비동기 프로그래밍을 하기 위한 콜백을 개선시키는 방법을 설명한다.
* 블로킹 연산을 스레드 풀에 덜어줌으로써 이벤트 루프를 블로킹하지 않는 방법을 알아본다.
* `asyncio` 서버를 작성하고, 웹 애플리케이션의 높은 동시성을 다시 생각해본다.
* `asyncio`가 파이썬 생태계에서 커다란 영향력을 줄 수밖에 없는 이유를 설명한다.

## 18.1 스레드와 코루틴의 비교

```python
# spinner_thread.py

import threading
import itertools
import time
import sys

class Signal: # 외부에서 스레드를 제어하기 위해 사용할 go 속성만 있는 가변 객체
    go = True

def spin(msg, signal):
    write, flush = sys.stdout.write, sys.stdout.flush
    for char in itertools.cycle('|/-\\'): # 무한 루프
        status = char + ' ' + msg
        write(status)
        flush()
        write('\x08' * len(status)) # 문자열 길이만큼 백스페이스 문자를
        # 반복해서 앞으로 이동 시킴
        time.sleep(.1)
        if not signal.go: # go == False 면 루프 종료
            break
    # 공백 문자로 덮어쓰고 다시 커서를 처음으로 이동해서 메세지 출력행 청소
    write(' ' * len(status) + '\x08' * len(status))

def slow_function():
    # 입출력을 위해 장시간 기다리는 것처럼 보이게 만든다.
    time.sleep(3)
    # 주 스레드에서 sleep() 함수를 호출할 때 GIL이 해제되므로
    # 두번째 스레드가 진행
    return 42

def supervisor(): 
    # 이 함수는 두번째 스레드를 만들고, 스레드 객체를 출력하고,
    # 시간이 오래 걸리는 연산을 수행하고 나서 스스로 스레드 제거
    signal = Signal()
    spinner = threading.Thread(target=spin,
                               args=('thinking!', signal))
    print('spinner object:', spinner) # 두번째 스레드 객체 출력
    spinner.start() # 두번째 스레드 실행
    result = slow_function() # slow_function() 실행
    # 그러면 주 스레드가 블로킹되고 그동안 두번째 스레드가 
    # 텍스트 스피너 애니메이션을 보여준다.
    signal.go = False # signal의 상태 변경
    spinner.join() # spinner 스레드가 끝날 때까지 기다린다
    return result

def main():
    result = supervisor() # supervisor() 실행
    print('Answer:', result)

if __name__ == '__main__':
    main()
```

```python
# spinner_asyncio.py

import asyncio
import itertools
import sys

@asyncio.coroutine # asyncio에 사용할 코루틴은 @asyncio.coroutine으로 데커레이트해야한다.
def spin(msg): # signal 인수 불필요
    write, flush = sys.stdout.write, sys.stdout.flush
    for char in itertools.cycle('|/-\\'):
        status = char + ' ' + msg
        write(status)
        flush()
        write('\x08' * len(status)) 
        try:
            yield from asyncio.sleep(.1) # 이벤트 루프를 블로킹하지 않고 잠자기 위해
            # time.sleep 대신 yield from asyncio.sleep 사용
        except asyncio.CancelledError: # spin()이 깨어난 후 
            # asyncio.CancelledError 예외가 발생하면 루프 종료
            break
    write(' ' * len(status) + '\x08' * len(status))

@asyncio.coroutine 
def slow_function():
    # slow_function()은 이제 코루틴으로서, 코루틴이 잠자면서 입출력을 수행하는 체 하는 동안
    # 이벤트 루프가 진행될 수 있게 하기 위해 yield from을 사용한다.
    yield from asyncio.sleep(3) # 메인 루프의 제어 흐름을 처리한다
    return 42

@asyncio.coroutine 
def supervisor(): 
    # supervisor()도 코루틴이므로 yield from을 이용해 slow_function() 구동 가능
    # asyncio.async()는 spin() 코루틴의 실행을 스케줄링하고 Task 객체 안에 넣어 즉시 반환
    spinner = asyncio.async(spin('thinking!')) 
    print('spinner object:', spinner) # Task 객체 출력
    result = yield from slow_function() # 함수를 구동해서 완료되면 반환된 값 가져온다
    # 그동안에 이벤트 루프는 계속 실행된다. slow_function()이 yield from asyncio.sleep()을 실행해서
    # 메인 루프로 제어권을 넘기기 때문이다.
    spinner.cancel() # Task객체는 cancel() 메서드를 호출해서 취소할 수 있다.
    # 그러면 코루틴이 중단된 곳의 yield from에서 asyncio.CancelledError 예외가 발생한다.
    # 코루틴은 예외를 잡아서 지연시키거나 취소 요청을 거부할 수 있다.
    return result

def main():
    loop = asyncio.get_event_loop() # 이벤트 루프에 대한 참조를 가져온다.
    result = loop.run_until_complete(supervisor()) # supervisor() 코루틴을 구동해서 완료한다.
    loop.close()
    print('Answer:', result)

if __name__ == '__main__':
    main()
```

```python
# 3.5 버전에 추가된 async와 await 키워드 이용한 구현

import asyncio
import itertools


async def spin(msg):  # <1>
    for char in itertools.cycle('|/-\\'):
        status = char + ' ' + msg
        print(status, flush=True, end='\r')
        try:
            await asyncio.sleep(.1)  # <2>
        except asyncio.CancelledError:  # <3>
            break
    print(' ' * len(status), end='\r')


async def slow_function():  # <4>
    # pretend waiting a long time for I/O
    await asyncio.sleep(3)  # <5>
    return 42


async def supervisor():  # <6>
    spinner = asyncio.create_task(spin('thinking!'))  # <7>
    print('spinner object:', spinner)  # <8>
    result = await slow_function()  # <9>
    spinner.cancel()  # <10>
    return result


def main():
    result = asyncio.run(supervisor())  # <11>
    print('Answer:', result)


if __name__ == '__main__':
    main()
```

반드시 `@asyncio.coroutine` 데커레이터를 사용해야 하는 것은 아니지만, 되도록이면 사용을 권고한다. 이는 이 데커레이터가 코루틴을 일반 함수와 다르게 보이도록 만들며, 코루틴이 yield from 되지 않고(즉 일부작업이 완료되지 않았으므로 버그 발생 가능성이 높음) 가비지 컬렉트되는 경우 경고 메시지를 출력하므로 디버깅에 도움이 된다.

두 예제의 `supervisor()` 함수의 주요 차이점은 다음과 같다.

* `asyncio.Task`는 `threading.Thread`와 거의 대등하다.
* `Task`는 코루틴을 구동하고, `Thread`는 콜러블을 호출한다.
* `Task` 객체는 직접 생성하지 않고, 코루틴을 `asyncio.async()`나 `loop.create_task()`에 전달해서 가져온다.
* `Task` 객체를 가져오면 이 객체는 이미 `asyncio.async()` 등에 의해 실행이 스케줄링되어 있다. `Thread` 객체는 `start()` 메서드를 호출해서 실행하라고 명령해야 한다.
* 스레드화된 `supervisor()`에서 `slow_function()`은 평범한 함수로서, 스레드에 의해 직접 호출된다. 비동기 `supervisor()`에서 `slow_function()`은 yield from으로 구동하는 코루틴이다.
* 스레드는 외부에서 API를 이용해서 중단시킬 수 없다. `Task`는 `cancel()` 메서드를 통해 CancelledError를 발생시키면 코루틴은 중단되었던 yield 문에서 예외를 잡아서 처리할 수 있다.
* `supervisor()` 코루틴은 `main()` 함수 안에서 `loop.run_until_complete()`로 실행해야 한다.

코루틴은 모든 것이 인터럽트로부터 보호된다. 여러 스레드의 연산을 동기화하기 위해 락을 잠그는 대신, 언제든 실행되고 있는 코루틴 중 하나만 사용하면 된다. 

### 18.1.1 asyncio.Future: 논블로킹 설계

`asyncio.Future`와 `concurrent.futures.Future` 클래스는 인터페이스가 거의 같지만 다르게 구현되어 있으므로 바꿔 쓸 수 없다. `asyncio.Future` 클래스도 `done()`, `add_done_callback()`, `result()` 등의 메서드를 제공한다. 앞 두개는 17.1.3절에서 설명한 대로 동작하지만 `result()`는 매우 다르며, 굳이 쓰지 말고 `yield from`을 쓰자...

* 코루틴 안에서 my_future가 실행을 완료한 다음에 수행할 작업은 단순히 yield from my_future 뒤에 넣으면 되므로 my_future.add_done_callback()을 호출할 필요가 없다.
* my_future에 대한 yield from 표현식의 값이 result가 되므로 my_future.result()를 호출할 필요가 없다.

### 18.1.2 Future, Task, 코루틴에서 생성하기

`foo()`가 코루틴 함수거나(즉, 호출되면 코루틴 객체 반환) Future나 Task 객체를 반환하는 일반 함수면 `res = yield from foo()` 코드가 작동한다. 코루틴을 받아서 Task 객체를 가져오기 위해서는 주로 다음 두 방법을 사용한다.

* `asyncio.async(coro_or_future, *, loop=None)` : 첫번째 인수가 Future나 Task 객체면 그대로 반환되며, 코루틴이면 `async()`가 `loop.create_task()`를 호출해서 Task를 생성한다. loop 키워드 인수에 이벤트 루프를 생략하면 `async()`가 `asyncio.get_event_loop()`를 호출해서 루프 객체를 가져온다.
* `BaseEventLoop.create_task(coro)` : 생략...

내부적으로 `asyncio.async()`를 이용해서 받은 코루틴을 자동으로 `asyncio.Task` 객체 안에 래핑하는 함수들이 많이 있으며, 대표적으로 `BaseEventLoop.run_until_complete()`다. 파이썬 콘솔에서 Future 객체나 코루틴을 실험하고 싶다면 다음 코드를 사용한다.

```python
import asyncio
def run_sync(coro_or_future):
    loop = asyncio.get_event_loop()
    return loop.run_until_complete(coro_or_future)

a = run_sync(some_coroutine())
```

## 18.2 asyncio와 aiohttp로 내려받기

asyncio는 TCP와 UDP만 직접 지원한다. HTTP 등의 프로토콜을 지원하려면 aiohttp 등의 서드파티 패키지를 사용해야 한다. 

국기 이미지를 내려받는 flag_asyncio.py 스크립트의 전체 흐름은 다음과 같다.

1. `download_one()`을 호출해서 생성된 여러 코루틴 객체를 이벤트 루프에 넣어 `download_many()` 안에서 프로세스를 시작한다.
1. `asyncio` 이벤트 루프는 각각의 코루틴을 차례대로 활성화한다.
1. `get_flag()` 등의 클라이언트 코루틴이 `aiohttp.request()` 등의 라이브러리 코루틴에 위임하기 위해 yield from을 사용하면, 제어권이 이벤트 루프로 넘어가서 이벤트 루프가 이전에 스케줄링된 다른 코루틴을 실행할 수 있게 된다.
1. 블로킹된 연산이 완료되었을 때 통지받기 위해 이벤트 루프는 콜백에 기반한 저수준 API를 사용한다.
1. 연산이 완료되면 메인 루프는 결과를 중단된 코루틴에 보낸다.
1. 그러고 나면 코루틴이 예를 들면 `get_flag()`의 `yield from resp.read()`와 같은 다음 yield from 문으로 넘어간다. 이제 이벤트 루프가 다시 제어권을 가져오고, 종료될 때까지 4,5,6단계를 반복한다.

```python
# flags_asyncio.py

import asyncio

import aiohttp  

from flags import BASE_URL, save_flag, show, main  # flags 모듈에서 구현한 일부 함수 재사용


@asyncio.coroutine  # 코루틴 데커레이트
def get_flag(cc):
    url = '{}/{cc}/{cc}.gif'.format(BASE_URL, cc=cc.lower())
    resp = yield from aiohttp.request('GET', url)  # 블로킹 연산은 코루틴으로 구현되었고 
    # yield from을 이용해서 이 코루틴에 위임하면 코루틴이 비동기식으로 실행된다.
    image = yield from resp.read()  # 응답 내용을 읽는 것은 별동의 비동기 연산에서 구현한다.
    return image


@asyncio.coroutine
def download_one(cc):  
    image = yield from get_flag(cc)  # 단어들을 yield from으로 가져온다는 것이 차이점
    show(cc)
    save_flag(image, cc.lower() + '.gif')
    return cc


def download_many(cc_list):
    loop = asyncio.get_event_loop()  # 하위 이벤트 루프 구현에 대한 참조를 가져온다.
    to_do = [download_one(cc) for cc in sorted(cc_list)]  # 국기 이미지를 가져올 때마다
    # download_one() 함수를 호출해서 제너레이터 객체 리스트를 생성한다.
    wait_coro = asyncio.wait(to_do)  # wait()는 블로킹 함수가 아니며, 일종의 코루틴으로
    # 자신에게 전달된 코루틴들이 모두 완료되면 완료된다.
    res, _ = loop.run_until_complete(wait_coro)  # wait_coro()가 완료될 떄까지 이벤트 루프 실행
    # 이 부분은 이벤트 루프가 실행하는 동안 블로킹된다
    loop.close() # 이벤트 루프 종료

    return len(res)


if __name__ == '__main__':
    main(download_many)
```

`asyncio.wait()` 코루틴은 Future 객체나 코루틴의 반복형을 받고, `wait()`는 각 코루틴을 Task 안에 래핑한다. `wait()`는 코루틴 함수이기 때문에 이를 호출하면 코루틴/제너레이터 객체가 반환되며, 이 코루틴을 구동하기 위해 `loop.run_until_complete()`에 전달한다. 실행이 완료되면 (<실행 완료된 Future들의 집합>, <실행이 완료되지 않은 Future들의 집합>) 튜플을 반환한다(위 예제에서는 두번째 집합은 공집합).

`asyncio`에 대해 알아야 할 새로운 개념이 많지만, `yield from` 키워드가 없는 것처럼 생각하면 이해하기 쉽다. 예를 들면 

```python
@asyncio.coroutine
def get_flag(cc):
    url = '{}/{cc}/{cc}.gif'.format(BASE_URL, cc=cc.lower())
    resp = yield from aiohttp.request('GET', url)
    image = yield from resp.read()
    return image
```

코드는 블로킹되지 않는다는 점만 제외하면 다음 코드와 똑같이 작동한다.

```python
def get_flag(cc):
    url = '{}/{cc}/{cc}.gif'.format(BASE_URL, cc=cc.lower())
    resp = aiohttp.request('GET', url)
    image = resp.read()
    return image
```

`yield from foo` 구문을 사용하면 현재의 코루틴(즉 yield from 코드가 있는 대표 제너레이터)이 중단되지만, 제어권이 이벤트 루프로 넘어가고 이벤트 루프가 다른 코루틴을 구동할 수 있게 되므로 블로킹되지 않는다. foo가 Future 객체이든 코루틴이든 이 객체가 완료되면 결과를 중단된 코루틴으로 반환해서 실행을 계속하게 만든다.

`asyncio`와 `yield from`을 사용할 때 다음과 같은 특징이 있다.

* 코루틴 체인은 가장 바깥쪽 대표 제너레이터를 `loop.run_until_complete()` 등의 asyncio API에 전달함으로써 구동된다. 
* 가장 안쪽 하위 제너레이터는 실제로 입출력을 수행하는 라이브러리 함수(ex: resp = yield from aiohttp.request('GET', url))여야 한다.

3.7 버전 코드는 다음과 같다.
```python

# BEGIN FLAGS_ASYNCIO
import os
import time
import sys
import asyncio  # <1>

import aiohttp  # <2>


POP20_CC = ('CN IN US ID BR PK NG BD RU JP '
            'MX PH VN ET EG DE IR TR CD FR').split()

BASE_URL = 'http://flupy.org/data/flags'

DEST_DIR = 'downloads/'


def save_flag(img, filename):
    path = os.path.join(DEST_DIR, filename)
    with open(path, 'wb') as fp:
        fp.write(img)


async def get_flag(session, cc):  # <3>
    url = '{}/{cc}/{cc}.gif'.format(BASE_URL, cc=cc.lower())
    async with session.get(url) as resp:        # <4>
        return await resp.read()  # <5>


def show(text):
    print(text, end=' ')
    sys.stdout.flush()


async def download_one(session, cc):  # <6>
    image = await get_flag(session, cc)  # <7>
    show(cc)
    save_flag(image, cc.lower() + '.gif')
    return cc


async def download_many(cc_list):
    async with aiohttp.ClientSession() as session:  # <8>
        res = await asyncio.gather(                 # <9>
            *[asyncio.create_task(download_one(session, cc))
                for cc in sorted(cc_list)])

    return len(res)


def main():  # <10>
    t0 = time.time()
    count = asyncio.run(download_many(POP20_CC))
    elapsed = time.time() - t0
    msg = '\n{} flags downloaded in {:.2f}s'
    print(msg.format(count, elapsed))


if __name__ == '__main__':
    main()
```

## 18.3 블로킹 호출을 에둘러 실행하기

Node.js의 창시자인 라이언 달은 블로킹 함수를 디스크나 네트워크 입출력의 수행으로 정의하면서, 이 함수들을 논블로킹 함수처럼 다루면 안 된다고 말한다. 이유는 다음 표를 보면 알 수 있다.

|장치|CPU 사이클 수|비례 체감 규모|
|:-:|:-:|:-:|
|L1 캐시|3|3초|
|L2 캐시|14|14초|
|램|250|250초|
|디스크|41,000,000|1.3년|
|네트워크|240,000,000|7.6년|

블로킹 함수가 전체 애플리케이션의 실행을 멈추지 않게 하는 두 가지 방법이 있다.

* 블로킹 연산을 각기 별도의 스레드에서 실행한다.
* 모든 블로킹 연산을 논블로킹 비동기 연산으로 바꾼다.

스레드는 제대로 작동하지만, 파이썬이 사용하는 OS 스레드는 각기 수 메가바이트의 메모리를 사용하므로, 수천 개의 연결을 처리해야 한다면 연결마다 하나의 스레드를 사용할 수 없다. 따라서 전통적으로 메모리 부담을 줄이기 위해 콜백으로 비동기 호출을 구현한다. 응답을 기다리는 대신 어떤 일이 발생할 때 호출될 함수를 등록한다. 이렇게 하면 우리가 호출한 것을 논블로킹으로 만들 수 있다.

따라서 flags_asyncio.py가 flags.py보다 5배나 빨리 실행되는 이유를 알 수 있다. flags.py는 이미지를 내려받을 때보다 수십억 CPU 사이클을 허비하기 때문이다. 실제로 CPI는 무언가 많은 일을 하지만, 우리가 만든 프로그램은 실행하지 않는다. 이와 반대로 flags_asyncio.py의 `download_many()` 함수가 `loop_until_complete`를 호출할 때, 이벤트 루프는 각각의 `download_one()` 코루틴을 첫 번째 yield from까지 구동시킨다. 이 yield from은 `get_flag()`의 첫 번째 yield from까지 구동시켜 `aiohttp.request()`를 호출한다. 이 과정에서 블로킹되는 부분이 없으므로 모든 요청이 아주 짧은 시간 안에 시작된다.

asyncio 기반 구조에서 첫 번째 응답을 받으면 이벤트 루프는 이것을 대기하고 있던 `get_flag()` 코루틴으로 전달한다. `get_flag()`가 응답을 받으면 다음번 yield from으로 넘어가서 `resp.read()`를 호출하고 제어권을 메인 루프로 넘긴다. 다른 요청도 거의 동시에 했기 때문에 각 응답이 거의 동시에 도착한다. 다른 요청도 거의 동시에 했기 때문에 각 응답이 거의 동시에 도착한다. 각 `get_flag()`이 반환됨에 따라 대표 제너레이터인 `download_one()`이 실행을 재개해서 이미지 파일을 저장한다.

## 18.4 asyncio 내려받기 스크립트 개선

앞서 17.5절에서 보았던 flags2 예제를 asyncio를 이용하여 개선해보자.

### 18.4.1 asyncio.as_completed() 사용하기

asyncio를 이용해서 flags2 예제를 구현하려면 concurrent.futures 버전이 재사용하는 여러 함수를 수정해야 한다. asyncio 프로그램에는 주 스레드가 하나만 있고 주 스레드에서 이벤트 루프를 실행하므로, 주 스레드 안에서 블로킹 함수를 호출하면 안 되기 때문이다. 

```python
# flags2_asyncio.py

import asyncio
import collections
import contextlib

import aiohttp
from aiohttp import web
import tqdm

from flags2_common import main, HTTPStatus, Result, save_flag

# 원격 서버에서 '503'과 같은 오류가 생기지 않도록 기본값을 낮게 설정한다.
DEFAULT_CONCUR_REQ = 5
MAX_CONCUR_REQ = 1000


class FetchError(Exception):  # 다른 HTTP와 네트워크 예외를 래핑하고 에러를 보고 하기 위해
    # 국가 코드를 보관하는 예외 클래스를 새로 정의한다.
    def __init__(self, country_code):
        self.country_code = country_code


@asyncio.coroutine
def get_flag(base_url, cc): 
    # get_flag()는 내려받은 이미지 파일의 크기를 바이트 단위로 넘겨주거나,
    # HTTP 응답 상태가 404일 때는 web.HTTPNotFound 예외를, 
    # 나머지 HTTP 에러 코드에 대해서는 aiohttp.HttpProcessingError 예외를 발생시킴
    url = '{}/{cc}/{cc}.gif'.format(base_url, cc=cc.lower())
    resp = yield from aiohttp.request('GET', url)
    with contextlib.closing(resp):
        if resp.status == 200:
            image = yield from resp.read()
            return image
        elif resp.status == 404:
            raise web.HTTPNotFound()
        else:
            raise aiohttp.HttpProcessingError(
                code=resp.status, message=resp.reason,
                headers=resp.headers)


@asyncio.coroutine
def download_one(cc, base_url, semaphore, verbose): 
    # semaphore 인수는 asyncio.Semephore 객체로서 동시 요청 수를 제한하기 위한 장치
    try:
        with (yield from semaphore):  # semaphore 카운터가 최대 허용 수에 이르렀을 때 코루틴 블로킹
            image = yield from get_flag(base_url, cc)  # 이 with문을 빠져나올 때 semaphore 카운터가 감수되고
            # 이 객체를 기다리고 있던 다른 코루틴 객체가 진행되도록 한다.
    except web.HTTPNotFound:  # 이미지를 찾을 수 없을 때는 HTTPStatus.not_found를 status로 설정
        status = HTTPStatus.not_found
        msg = 'not found'
    except Exception as exc: # 그외 나머지 예외는 국가 코드 담은 FetchErrror 반환
        raise FetchError(cc) from exc  
    else:
        save_flag(image, cc.lower() + '.gif')  # 이 함수가 실제로 국기 이미지를 디스크에 저장
        status = HTTPStatus.ok
        msg = 'OK'

    if verbose and msg:
        print(cc, msg)

    return Result(status, cc)
# END FLAGS2_ASYNCIO_TOP

# BEGIN FLAGS2_ASYNCIO_DOWNLOAD_MANY
@asyncio.coroutine
def downloader_coro(cc_list, base_url, verbose, concur_req):  # 코루틴은 download_many()와 동일한 인수를 받지만
    # 일반 함수가 아니라 코루틴 함수이므로 main()에서 바로 호출할 수 없다.
    counter = collections.Counter()
    semaphore = asyncio.Semaphore(concur_req)  # 동시에 concure_seq개까지의 코루틴을 실행할 수 있게 해주는
    # 세마포어 생성
    to_do = [download_one(cc, base_url, semaphore, verbose)
             for cc in sorted(cc_list)]  # 코루틴 객체 리스트 생성

    to_do_iter = asyncio.as_completed(to_do)  # 실행이 완료된 Future 객체를 반환하는 반복자
    if not verbose:
        to_do_iter = tqdm.tqdm(to_do_iter, total=len(cc_list))  # 반복자를 tqdm() 하무에 래핑해서 진행 상태를 출력함
    for future in to_do_iter:  # 완료된 Future 객체들을 반복한다.
        try:
            res = yield from future  # asyncio.Future 객체의 결과를 가져올 때는 객체의 result() 메서드를 호출하는 것보다
            # yield from 을 사용하는 것이 더 쉽다.
        except FetchError as exc:  # download_one()에서 발생하는 모든 예외는 FetchError 객체 안에 래핑되어 있다.
            country_code = exc.country_code  # FetchError 예외 안에서 에러가 발생한 국가 코드를 가져옴
            try:
                error_msg = exc.__cause__.args[0]  # 원래 예외에서 오류 메시지를 가져오려 시도한다.
            except IndexError:
                error_msg = exc.__cause__.__class__.__name__  # 원래 예외에서 에러 메시지를 발견할 수 없으면
                # 연결된 예외 클래스명을 에러 메세지로 사용한다.
            if verbose and error_msg:
                msg = '*** Error for {}: {}'
                print(msg.format(country_code, error_msg))
            status = HTTPStatus.error
        else:
            status = res.status

        counter[status] += 1  # 결과를 합계에 추가한다.

    return counter  # 다른 버전의 프로그램과 마찬가지로 카운터 반환


def download_many(cc_list, base_url, verbose, concur_req):
    loop = asyncio.get_event_loop()
    coro = downloader_coro(cc_list, base_url, verbose, concur_req)
    counts = loop.run_until_complete(coro)  
    loop.close()  # 모든 작업이 완료되면 이벤트 루프를 종료하고 카운터 반환

    return counts


if __name__ == '__main__':
    main(download_many, DEFAULT_CONCUR_REQ, MAX_CONCUR_REQ)
```

```python
# flags2_asyncio_3.7.py

import asyncio
import collections

import aiohttp
from aiohttp import web
import tqdm

from flags2_common import main, HTTPStatus, Result, save_flag

# default set low to avoid errors from remote site, such as
# 503 - Service Temporarily Unavailable
DEFAULT_CONCUR_REQ = 5
MAX_CONCUR_REQ = 1000


class FetchError(Exception):  # <1>
    def __init__(self, country_code):
        self.country_code = country_code


async def get_flag(session, base_url, cc): # <2>
    url = '{}/{cc}/{cc}.gif'.format(base_url, cc=cc.lower())
    async with session.get(url) as resp:
        if resp.status == 200:
            return await resp.read()
        elif resp.status == 404:
            raise web.HTTPNotFound()
        else:
            raise aiohttp.HttpProcessingError(
                code=resp.status, message=resp.reason,
                headers=resp.headers)


async def download_one(session, cc, base_url, semaphore, verbose):  # <3>
    try:
        async with semaphore:  # <4>
            image = await get_flag(session, base_url, cc)  # <5>
    except web.HTTPNotFound:  # <6>
        status = HTTPStatus.not_found
        msg = 'not found'
    except Exception as exc:
        raise FetchError(cc) from exc  # <7>
    else:
        save_flag(image, cc.lower() + '.gif')  # <8>
        status = HTTPStatus.ok
        msg = 'OK'

    if verbose and msg:
        print(cc, msg)

    return Result(status, cc)
# END FLAGS2_ASYNCIO_TOP

# BEGIN FLAGS2_ASYNCIO_DOWNLOAD_MANY
async def downloader_coro(cc_list, base_url, verbose, concur_req):  # <1>
    counter = collections.Counter()
    semaphore = asyncio.Semaphore(concur_req)  # <2>
    async with aiohttp.ClientSession() as session:  # <8>
        to_do = [download_one(session, cc, base_url, semaphore, verbose)
                for cc in sorted(cc_list)]  # <3>

        to_do_iter = asyncio.as_completed(to_do)  # <4>
        if not verbose:
            to_do_iter = tqdm.tqdm(to_do_iter, total=len(cc_list))  # <5>
        for future in to_do_iter:  # <6>
            try:
                res = await future  # <7>
            except FetchError as exc:  # <8>
                country_code = exc.country_code  # <9>
                try:
                    error_msg = exc.__cause__.args[0]  # <10>
                except IndexError:
                    error_msg = exc.__cause__.__class__.__name__  # <11>
                if verbose and error_msg:
                    msg = '*** Error for {}: {}'
                    print(msg.format(country_code, error_msg))
                status = HTTPStatus.error
            else:
                status = res.status

            counter[status] += 1  # <12>

    return counter  # <13>


def download_many(cc_list, base_url, verbose, concur_req):
    loop = asyncio.get_event_loop()
    coro = downloader_coro(cc_list, base_url, verbose, concur_req)
    counts = loop.run_until_complete(coro)  # <14>
    loop.close()  # <15>

    return counts


if __name__ == '__main__':
    main(download_many, DEFAULT_CONCUR_REQ, MAX_CONCUR_REQ)
```

### 18.4.2 Executor를 이용해서 이벤트 루프 블로킹 피하기

앞선 예제에서는 `save_flag()`가 블로킹 함수다. 이 함수가 asyncio 이벤트 루프와 공유한 유일한 스레드를 블로킹하므로, 파일을 저장하는 동안 애플리케이션 전체가 멈춘다. 이 문제의 해결책은 이벤트 루프 객체의 `run_in_executor()` 메서드다.

asyncio 이벤트 루프는 스레드 풀 실행자를 내부에 가지고 있으며 `run_in_executor()` 메서드에 실행할 콜러블을 전달할 수 있다. 다음 예제처럼 `download_one()` 코루틴에서 몇 줄만 수정하면 된다.

```python
@asyncio.coroutine
def download_one(cc, base_url, semaphore, verbose):
    try:
        with (yield from semaphore):
            image = yield from get_flag(base_url, cc)
    except web.HTTPNotFound:
        status = HTTPStatus.not_found
        msg = 'not found'
    except Exception as exc:
        raise FetchError(cc) from exc
    else:
        loop = asyncio.get_event_loop()  # 이벤트 루프 가져옴
        loop.run_in_executor(None,  # 첫번째 인수는 실행자 객체. 이벤트 루프의 기본 스레드 풀 실행자를 사용할 때는 None 지정
                save_flag, image, cc.lower() + '.gif')  # 나머지 인수는 콜러블 및 콜러블이 받을 위치 인수
        status = HTTPStatus.ok
        msg = 'OK'

    if verbose and msg:
        print(cc, msg)

    return Result(status, cc)
```

3.7 버전은 다음과 같다.

```python
import asyncio
import collections

import aiohttp
from aiohttp import web
import tqdm

from flags2_common import main, HTTPStatus, Result, save_flag

# default set low to avoid errors from remote site, such as
# 503 - Service Temporarily Unavailable
DEFAULT_CONCUR_REQ = 5
MAX_CONCUR_REQ = 1000


class FetchError(Exception):  # <1>
    def __init__(self, country_code):
        self.country_code = country_code


async def get_flag(session, base_url, cc): # <2>
    url = '{}/{cc}/{cc}.gif'.format(base_url, cc=cc.lower())
    async with session.get(url) as resp:
        if resp.status == 200:
            return await resp.read()
        elif resp.status == 404:
            raise web.HTTPNotFound()
        else:
            raise aiohttp.HttpProcessingError(
                code=resp.status, message=resp.reason,
                headers=resp.headers)


async def download_one(session, cc, base_url, semaphore, verbose):  # <3>
    try:
        async with semaphore:  # <4>
            image = await get_flag(session, base_url, cc)  # <5>
    except web.HTTPNotFound:  # <6>
        status = HTTPStatus.not_found
        msg = 'not found'
    except Exception as exc:
        raise FetchError(cc) from exc  # <7>
    else:
        save_flag(image, cc.lower() + '.gif')  # <8>
        status = HTTPStatus.ok
        msg = 'OK'

    if verbose and msg:
        print(cc, msg)

    return Result(status, cc)
# END FLAGS2_ASYNCIO_TOP

# BEGIN FLAGS2_ASYNCIO_DOWNLOAD_MANY
async def downloader_coro(cc_list, base_url, verbose, concur_req):  # <1>
    counter = collections.Counter()
    semaphore = asyncio.Semaphore(concur_req)  # <2>
    async with aiohttp.ClientSession() as session:  # <8>
        to_do = [download_one(session, cc, base_url, semaphore, verbose)
                for cc in sorted(cc_list)]  # <3>

        to_do_iter = asyncio.as_completed(to_do)  # <4>
        if not verbose:
            to_do_iter = tqdm.tqdm(to_do_iter, total=len(cc_list))  # <5>
        for future in to_do_iter:  # <6>
            try:
                res = await future  # <7>
            except FetchError as exc:  # <8>
                country_code = exc.country_code  # <9>
                try:
                    error_msg = exc.__cause__.args[0]  # <10>
                except IndexError:
                    error_msg = exc.__cause__.__class__.__name__  # <11>
                if verbose and error_msg:
                    msg = '*** Error for {}: {}'
                    print(msg.format(country_code, error_msg))
                status = HTTPStatus.error
            else:
                status = res.status

            counter[status] += 1  # <12>

    return counter  # <13>


def download_many(cc_list, base_url, verbose, concur_req):
    loop = asyncio.get_event_loop()
    coro = downloader_coro(cc_list, base_url, verbose, concur_req)
    counts = loop.run_until_complete(coro)  # <14>
    loop.close()  # <15>

    return counts


if __name__ == '__main__':
    main(download_many, DEFAULT_CONCUR_REQ, MAX_CONCUR_REQ)
```

## 18.5 콜백에서 Future와 코루틴으로

코루틴이 고전적인 콜백 스타일을 어떻게 향상시키는지 명확히 알아두는 것이 좋다. 콜백 지옥은 어떤 연산이 다른 연산에 종속적일 때 콜백 안에 콜백이 들어가는 것을 말한다.

```python
def stage1(response1):
    request2 = step1(response1)
    api_call2(request2, stage2)

def stage2(response2):
    request3 = step2(response2)
    api_call3(request3, stage3)
    
def stage3(response3):
    step3(response3)
    
api_call1(request1, stage1)
```

이런 코드는 작성하기도, 읽기도 어렵다. 또한 지역 변수는 사용 후 사라지게 된다. 이런 상황에서는 코루틴이 도움이 된다. 하나의 함수 안에서 전체 작업의 콘텍스트를 지역 변수 안에 넣고, 일반적인 함수를 호출하듯이 연속으로 세 개의 함수를 호출하면 되기 때문이다.

```python
@asyncio.coroutine
def three_stages(request1):
    response1 = yield from api_call1(request1)
    # 1단계
    request2 = step1(response1)
    response2 = yield from api_call2(request2)
    # 2단계
    request3 = step2(response2)
    response3 = yield from api_call3(request3)
    # 3단계
    step3(response3)
   
loop.create_task(three_stages(request1)) # 명시적으로 실행을 스케줄링
```

이런 코드는 더 이해하기 쉬우며 앞 단계의 결과나 지역 변수들도 사용할 수 있다. 예외 처리도 좀 더 쉽긴 하지만, 평범하게 함수들을 호출할 수는 없다.

### 18.5.1 한 번 내려받을 때 여러 가지 요청하기

단순히 국가 코드만 사용하는 대신 국가 코드와 국가명을 사용해서 각 나라의 국기를 저장하는 경우를 생각해보자. 이제는 이미지 뿐 아니라 국가명이 들어 있는 파일에서 가져와야 하므로 이미지 하나마다 HTTP 요청을 두 번 해야 한다.

스레드 버전에서는 동일 작업 안에서 여러 요청을 명시하기 쉽다. 한번 요청한 후 다음 번 요청을 하면서 스레드를 두 번 블로킹하고 두 가지 데이터를 지역 변수에 저장해서 파일을 저장할 때 사용하면 된다. 이를 콜백으로 구현하면 까다로우므로, 코루틴과 yield from을 사용하면 조금 나아진다.

아래는 비동기식으로 국기를 내려받는 세 번째 버전으로, 국가명으로 각각의 국기 파일을 저장한다. 다음 함수가 수정되었다.

* download_one() : 이제 이 코루틴은 yield from을 사용해서 `get_flag()` 및 새로 만들어진 `get_country()` 코루틴에 위임
* get_flag() : 이 코루틴에 들어 있던 코드 대부분은 `get_country()`에서도 사용할 수 있도록 `http_get()` 코루틴으로 이동
* get_country() : 이 코루틴은 국가 코드에 대한 metadata.json 파일을 가져와서 국가명을 꺼낸다.
* http_get() : 웹에서 파일을 가져오기 위해 공통으로 사용하는 코드

```python
@asyncio.coroutine
def http_get(url):
    res = yield from aiohttp.request('GET', url)
    if res.status == 200:
        ctype = res.headers.get('Content-type', '').lower()
        if 'json' in ctype or url.endswith('json'):
            data = yield from res.json()  # json 이 파일명에 있거나 자료형이면 json() 이용하여 파싱
        else:
            data = yield from res.read()  # 그렇지 않으면 read() 호출해서 바이트 그대로 가져옴
        return data

    elif res.status == 404:
        raise web.HTTPNotFound()
    else:
        raise aiohttp.errors.HttpProcessingError(
            code=res.status, message=res.reason,
            headers=res.headers)


@asyncio.coroutine
def get_country(base_url, cc):
    url = '{}/{cc}/metadata.json'.format(base_url, cc=cc.lower())
    metadata = yield from http_get(url)  # metadata는 JSON 콘텐츠로 만들어진 딕셔너리 객체를 받는다
    return metadata['country']


@asyncio.coroutine
def get_flag(base_url, cc):
    url = '{}/{cc}/{cc}.gif'.format(base_url, cc=cc.lower())
    return (yield from http_get(url)) # return yield from처럼 키워드가 나란히 나오면 파이썬 파서가 혼동해서 오류남


@asyncio.coroutine
def download_one(cc, base_url, semaphore, verbose):
    try:
        with (yield from semaphore): # 가능한 빨리 가져오기 위해 semaphore에 의해 제어되는 
            # 두개의 with 블록에서 get_flag()와 get_country()를 따로 호출한다.
            image = yield from get_flag(base_url, cc)
        with (yield from semaphore):
            country = yield from get_country(base_url, cc)
    except web.HTTPNotFound:
        status = HTTPStatus.not_found
        msg = 'not found'
    except Exception as exc:
        raise FetchError(cc) from exc
    else:
        country = country.replace(' ', '_')
        filename = '{}-{}.gif'.format(country, cc)
        loop = asyncio.get_event_loop()
        loop.run_in_executor(None, save_flag, image, filename)
        status = HTTPStatus.ok
        msg = 'OK'

    if verbose and msg:
        print(cc, msg)

    return Result(status, cc)
```

yield from을 사용해야할 때와 사용할 수 없을 때를 구분하는 것이 관건이다. 코루틴 및 Task 객체 등의 asyncio.Future 객체에는 yield from을 사용해야 한다. 

## 18.6 asyncio 서버 작성

이건 좀... too much...